In [1]:
pip install xgboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [16]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

## Enginering features
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates', 'fixed acidity']]
Y = train['quality'] 

test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates', 'fixed acidity']]

# Optimal Rounder

In [17]:
from functools import partial
import numpy as np
import scipy as sp

class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8

        ll = cohen_kappa_score(y, X_p, weights = 'quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [3.5, 4.5, 5.5, 6.5, 7.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method = 'nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 3
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 4
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 5
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 6
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 7
            else:
                X_p[i] = 8
        return X_p

    def coefficients(self):
        return self.coef_['x']

# Optuna Optimzation

In [18]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(objective = 'reg:absoluteerror',
                 eval_metric = 'mae',
                 tree_method = 'hist', 
                 random_state = 42,
                 max_depth = trial.suggest_int('max_depth', 2, 10),
                 learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log = True),
                 n_estimators = trial.suggest_int('n_estimators', 30, 10000),
                 gamma = trial.suggest_float('gamma', 0, 10),
                 min_child_weight = trial.suggest_int('min_child_weight', 1, 100),
                 colsample_bytree = trial.suggest_float('colsample_bytree', 0.2, 0.9),
                 subsample = trial.suggest_float('subsample', 0.2, 0.9)
                )

    scores = []

#     skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)
    skf = KFold(n_splits = 5, shuffle = True, random_state = 42)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        Y_train , Y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        model = XGBRegressor(**param).fit(X_train, Y_train)

        preds_train = model.predict(X_train)
        preds_valid = model.predict(X_valid)
        
        optR = OptimizedRounder()
        optR.fit(preds_train, Y_train)
        coef = optR.coefficients()
        preds_valid = optR.predict(preds_valid, coef).astype(int)
        
        score = cohen_kappa_score(Y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
        
    return np.mean(scores)

In [20]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-08 17:22:18,222] A new study created in memory with name: no-name-f5172b81-eb87-42dd-9f69-53969e58d581


0 1 2 3 4 

[I 2023-02-08 17:22:40,268] Trial 0 finished with value: 0.450832755964794 and parameters: {'max_depth': 5, 'learning_rate': 0.0020921514749607075, 'n_estimators': 410, 'gamma': 1.1407010403628104, 'min_child_weight': 42, 'colsample_bytree': 0.7415141958333946, 'subsample': 0.7049778584246975}. Best is trial 0 with value: 0.450832755964794.


0 1 2 3 4 

[I 2023-02-08 17:23:04,569] Trial 1 finished with value: 0.3017280606794285 and parameters: {'max_depth': 3, 'learning_rate': 0.0005144795239126138, 'n_estimators': 2295, 'gamma': 4.350703448429015, 'min_child_weight': 35, 'colsample_bytree': 0.7129661958116993, 'subsample': 0.26497537726195786}. Best is trial 0 with value: 0.450832755964794.


...

[I 2023-02-08 17:45:42,361] Trial 47 finished with value: 0.504642070466378 and parameters: {'max_depth': 7, 'learning_rate': 0.02574549384071337, 'n_estimators': 1517, 'gamma': 5.039489439751967, 'min_child_weight': 97, 'colsample_bytree': 0.235468754032218, 'subsample': 0.6611316409402814}. Best is trial 44 with value: 0.530432165008472.


0 1 2 3 4 

[I 2023-02-08 17:46:19,190] Trial 48 finished with value: 0.49495306333120104 and parameters: {'max_depth': 6, 'learning_rate': 0.06552301605033681, 'n_estimators': 2108, 'gamma': 4.137852017727796, 'min_child_weight': 45, 'colsample_bytree': 0.3307307522583229, 'subsample': 0.5758303625100999}. Best is trial 44 with value: 0.530432165008472.


0 1 2 3 4 

[I 2023-02-08 17:46:39,814] Trial 49 finished with value: 0.46689890820811986 and parameters: {'max_depth': 9, 'learning_rate': 0.03594590358675309, 'n_estimators': 523, 'gamma': 6.104786108515054, 'min_child_weight': 82, 'colsample_bytree': 0.2828770677063024, 'subsample': 0.7088671264589825}. Best is trial 44 with value: 0.530432165008472.


In [21]:
study.best_trial.value

0.530432165008472

In [22]:
study.best_trial.params

{'max_depth': 7,
 'learning_rate': 0.037306935544411675,
 'n_estimators': 711,
 'gamma': 6.424280577329913,
 'min_child_weight': 100,
 'colsample_bytree': 0.24827297782231844,
 'subsample': 0.619874598507635}

# XGBoost Modeling

In [87]:
XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
# for i in range(5):
    
#     skf = StratifiedKFold(n_splits = 5, random_state = 40, shuffle = True)
skf = KFold(n_splits = 10, random_state = 42, shuffle = True)
    
for train_ix, test_ix in skf.split(X, Y):
        
    ## Splitting the data 
    X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
    Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
    ## Building RF model
    XGB_md = XGBRegressor(**study.best_trial.params, random_state = 0).fit(X_train, Y_train)
#     XGB_md = XGBRegressor(tree_method = 'hist',
#                           colsample_bytree = 0.2715039879238372, 
#                           gamma = 8.977910227926444, 
#                           learning_rate = 0.048945719202738824, 
#                           max_depth = 6, 
#                           min_child_weight = 71, 
#                           n_estimators = 1189, 
#                           subsample = 0.739476345973539, 
#                           random_state = 42).fit(X_train, Y_train)
    XGB_imp.append(XGB_md.feature_importances_)
        
    ## Predicting on X_test and test
    XGB_pred_1 = XGB_md.predict(X_test)
    XGB_pred_2 = XGB_md.predict(test_md)
        
    ## Applying Optimal Rounder (using abhishek approach)
    optR = OptimizedRounder()
    optR.fit(XGB_md.predict(X_train), Y_train)
    coef = optR.coefficients()
    XGB_pred_1 = optR.predict(XGB_pred_1, coef).astype(int)
    XGB_pred_2 = optR.predict(XGB_pred_2, coef).astype(int)
        
    ## Computing weighted quadratic kappa
    XGB_cv_scores.append(cohen_kappa_score(Y_test, XGB_pred_1, weights = 'quadratic'))
    preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

The average roc-auc score over 5-folds (run 5 times) is: 0.5415626356860228


In [80]:
a = [0, 1, 2, 3]
np.argmax(a)

3

In [81]:
a[3]

3

In [85]:
x = [i for i in range(5)]
np.argmax(x)

4

In [86]:
x

[0, 1, 2, 3, 4]

In [78]:
XGB_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = XGB_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [79]:
submission['quality'].value_counts()

5    602
6    443
7    327
Name: quality, dtype: int64

In [75]:
submission['quality'].value_counts()

5    634
6    410
7    328
Name: quality, dtype: int64

In [76]:
submission.to_csv('XGB_Reg_FE_6.csv', index = False)

In [10]:
submission['quality'].value_counts()

5    622
6    417
7    333
Name: quality, dtype: int64

In [11]:
submission.to_csv('XGB_Reg_FE_5.csv', index = False)

In [40]:
XGB_md = XGBRegressor(**study.best_trial.params, 
                      random_state = 4).fit(X, Y)
# XGB_md = XGBRegressor(tree_method = 'hist',
#                       colsample_bytree = 0.2715039879238372, 
#                       gamma = 8.977910227926444, 
#                       learning_rate = 0.048945719202738824, 
#                       max_depth = 6, 
#                       min_child_weight = 71, 
#                       n_estimators = 1189, 
#                       subsample = 0.739476345973539, 
#                       random_state = 4).fit(X, Y)

optR = OptimizedRounder()
optR.fit(XGB_md.predict(X), Y)
coef = optR.coefficients()
XGB_pred = XGB_md.predict(test_md)
XGB_pred = optR.predict(XGB_pred, coef).astype(int)

In [41]:
submission['quality'] = XGB_pred
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [88]:
file_name = 'XGB_Reg_FUll_Seed_' + str(1) + '.csv'
file_name

'XGB_Reg_FUll_Seed_1.csv'

In [39]:
## seed = 42
submission['quality'].value_counts()

5    594
6    454
7    324
Name: quality, dtype: int64

In [42]:
## seed 4
submission['quality'].value_counts()

5    671
6    380
7    321
Name: quality, dtype: int64

In [15]:
submission.to_csv('XGB_Reg_full_FE_6.csv', index = False)

# Modeling XGBoost Like Crazy

In [48]:
XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
for i in range(100):
    
    print(i)
    ## Building RF model
    XGB_md = XGBRegressor(**study.best_trial.params, 
                          random_state = i).fit(X, Y)
    
#     XGB_imp.append(XGB_md.feature_importances_)
        
    ## Predicting on X_test and test
    XGB_pred_1 = XGB_md.predict(X)
    XGB_pred_2 = XGB_md.predict(test_md)
    
    optR = OptimizedRounder()
    optR.fit(XGB_pred_1, Y)
    coef = optR.coefficients()
    XGB_pred_1 = optR.predict(XGB_pred_1, coef).astype(int)
    XGB_pred_2 = optR.predict(XGB_pred_2, coef).astype(int)
    
    ## Computing roc-auc score
    XGB_cv_scores.append(cohen_kappa_score(Y, XGB_pred_1, weights = 'quadratic'))
    preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
The average roc-auc score over 5-folds (run 5 times) is: 0.5650031992255752


In [49]:
 pd.DataFrame(preds)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,1322,1323,1324,1325,1326,1327,1328,1329,1330,1331,1332,1333,1334,1335,1336,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346,1347,1348,1349,1350,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371
0,5,6,5,6,6,6,6,6,6,5,5,6,7,6,6,5,7,5,5,7,7,6,5,7,5,5,7,5,7,5,5,7,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,6,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,5,7,5,5,5
1,5,6,5,6,6,6,5,6,6,5,5,6,7,6,6,5,7,5,5,7,7,6,5,7,5,5,7,5,7,5,5,7,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,5,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,5,7,5,5,5
2,5,6,5,7,6,6,5,6,6,5,5,6,7,6,6,5,7,5,5,7,7,6,5,7,5,5,7,5,7,5,5,7,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,5,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,5,7,5,5,5
3,5,6,5,7,6,6,6,6,6,6,5,6,7,6,6,5,7,5,5,7,7,6,6,7,5,5,7,6,7,6,5,7,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,6,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,6,7,5,5,5
4,5,6,5,6,6,6,5,6,6,5,5,6,6,6,6,5,7,5,5,7,7,6,5,7,5,5,7,5,7,5,5,6,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,5,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,5,7,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,6,5,7,6,6,5,6,6,6,5,6,7,6,6,5,7,5,5,7,7,6,6,7,5,5,7,5,7,5,5,7,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,6,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,5,7,5,5,5
96,5,6,5,6,6,6,6,6,6,6,5,6,7,6,6,5,7,5,5,7,7,6,6,7,5,5,7,6,7,5,5,7,5,5,6,6,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,6,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,6,7,5,5,5
97,5,6,5,6,6,6,5,6,6,5,5,6,6,6,6,5,7,5,5,7,7,6,5,7,5,5,7,5,7,5,5,6,5,5,6,5,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,5,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,5,7,5,5,5
98,5,6,5,6,6,6,6,6,6,6,5,6,6,6,6,5,7,5,5,7,7,6,6,7,5,5,7,5,7,6,5,7,5,5,6,6,7,6,5,5,5,5,5,7,7,6,5,5,6,5,...,6,7,6,5,5,5,5,5,5,5,5,7,7,5,6,6,5,6,6,5,7,5,5,6,7,7,5,6,6,5,5,6,5,5,5,5,5,7,7,7,6,5,5,7,7,6,7,5,5,5


In [50]:
XGB_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ]

submission['quality'] = XGB_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [51]:
submission['quality'].value_counts()

5    652
6    392
7    328
Name: quality, dtype: int64

In [52]:
submission.to_csv('XGB_Reg_full_100_FE_7.csv', index = False)